# AML CLI v2 と YAML による学習

[00_LoadData.ipynb](https://github.com/notanaha/oh4ml-lite/blob/main/00_LoadData.ipynb) を実行して、あらかじめデータセットの登録をおこなってください。

> Note : データセットのアップロード・登録を CLI v2 と YAML で記述できます。

## Azure へのログイン

In [ ]:
!az login

In [2]:
!az account set -s {AZURE_SUBSCRIPTION_ID}

## AML ワークスペースの環境設定

In [3]:
my_resource_group = "{AML-RESOURCE-GROUP-NAME}"
my_workspace = "{AML-WORSPACE-NAME}"

## [Optional] AML コンピュート (クラスター) の作成

demo-cpucluster1 の名前の AML compute cluster がない場合には、下記を実行<br>
(既に存在する場合には実行不要)

In [5]:
!az ml compute create --name demo-cpucluster1 \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace \
  --type amlcompute \
  --min-instances 0 \
  --max-instances 1 \
  --size Standard_D2_v2

Command group 'ml compute' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "id": "/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST20220331-AML/providers/Microsoft.MachineLearningServices/workspaces/ws01/computes/demo-cpucluster1",
  "idle_time_before_scale_down": 120,
  "location": "eastus",
  "max_instances": 1,
  "min_instances": 0,
  "name": "demo-cpucluster1",
  "network_settings": {},
  "provisioning_state": "Succeeded",
  "resourceGroup": "TEST20220331-AML",
  "size": "STANDARD_D2_V2",
  "ssh_public_access_enabled": true,
  "tier": "dedicated",
  "type": "amlcompute"
}


## [Optional] Environment (環境) の作成

arima-env の Environment (環境) を AML 上に作成していない場合には下記を実行。<br>
(既に作成済の場合は不要)

In [6]:
%%writefile arima-env-config.yml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: arima-env
image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
conda_file: arima-env.yml
description: This is example

Writing arima-env-config.yml


In [7]:
!az ml environment create --file arima-env-config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml environment' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "conda_file": {
    "channels": [
      "anaconda",
      "conda-forge"
    ],
    "dependencies": [
      "python=3.6.9",
      {
        "pip": [
          "azureml-defaults",
          "statsmodels==0.12.2",
          "scipy",
          "scikit-learn",
          "joblib",
          "numpy",
          "pandas",
          "matplotlib",
          "seaborn",
          "numpy"
        ]
      }
    ],
    "name": "azureml_669dcef02b953becb05c12c0f43bc7d3"
  },
  "creation_context": {
    "created_at": "2022-03-31T06:52:49.470879+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User",
    "last_modified_at": "2022-03-31T06:52:49.470879+00:00",
    "last_modified_by": "Tsuyoshi Matsuzaki",
    "last_modified_by_type": "User"
  },
  "description": "This is example",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resou

## 学習実行

In [10]:
%%writefile train-job-config.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: 
  local_path: scripts
command: >-
  python trainAndRegisterModel.py
environment: azureml:arima-env:1
compute: azureml:demo-cpucluster1
display_name: arima-mlops-remote-cli
experiment_name: arima-mlops-remote-cli
description: Train demo

Overwriting train-job-config.yml


In [12]:
!az ml job create --file train-job-config.yml \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace

Command group 'ml job' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
{
  "code": "azureml:6227ec3a-5038-4a58-8ba1-a8c1d0a99d7a:1",
  "command": "python trainAndRegisterModel.py",
  "compute": "azureml:demo-cpucluster1",
  "creation_context": {
    "created_at": "2022-03-31T07:26:29.483158+00:00",
    "created_by": "Tsuyoshi Matsuzaki",
    "created_by_type": "User"
  },
  "description": "Train demo",
  "display_name": "arima-mlops-remote-cli",
  "environment": "azureml:arima-env:1",
  "environment_variables": {},
  "experiment_name": "arima-mlops-remote-cli",
  "id": "azureml:/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourceGroups/TEST20220331-AML/providers/Microsoft.MachineLearningServices/workspaces/ws01/jobs/5ed91d8f-55e8-4382-a0a1-d4e7ad4078b0",
  "inputs": {},
  "name": "5ed91d8f-55e8-4382-a0a1-d4e7ad4078b0",
  "outputs": {
    "default": {
      "folder": "azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.5

## 補足説明

トレーニングとモデルの登録をわけて、すべて CLI v2 + YAML で記述することもできます。<br>
下記は、CLI v2 によるモデル登録のサンプルコマンドです。

```
!az ml model create --name test-model \
  --version 1 \
  --local-path ./generated_model \
  --resource-group $my_resource_group \
  --workspace-name $my_workspace
```